In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

In [2]:

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Train a logistic regression model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

LogisticRegression(max_iter=200)

In [4]:

# Make predictions
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Model Accuracy: {accuracy:.2f}")

# Predict on new data
new_sample = [[5.1, 3.5, 1.4, 0.2]]  # Example new input
predicted_class = model.predict(new_sample)
print(f"🚀 Predicted Class: {predicted_class[0]}")

✅ Model Accuracy: 1.00
🚀 Predicted Class: 0


In [5]:
import joblib

# Save the model
joblib.dump(model, "test_lr.pkl")
print("✅ Model saved successfully!")

# Load the model
loaded_model = joblib.load("test_lr.pkl")

# Predict with the loaded model
new_sample = [[5.1, 3.5, 1.4, 0.2]]
predicted_class = loaded_model.predict(new_sample)
print(f"🚀 Predicted Class: {predicted_class[0]}")


✅ Model saved successfully!
🚀 Predicted Class: 0


In [9]:
from skl2onnx import to_onnx
import numpy as np

# Load trained model
clf = joblib.load("test_lr.pkl")

# Convert to ONNX
X_sample = np.array([[5.1, 3.5, 1.4, 0.2]], dtype=np.float32)  # Sample input
onx = to_onnx(clf, X_sample, target_opset=12)

# Save ONNX model
with open("test_lr.onnx", "wb") as f:
    f.write(onx.SerializeToString())

print("✅ Model converted and saved as logistic_regression.onnx")

✅ Model converted and saved as logistic_regression.onnx


In [ ]:
from huggingface_hub import login

hf_token = ""

login(token=hf_token)

print("✅ Successfully logged in to the Hugging Face Hub!")

✅ Successfully logged in to the Hugging Face Hub!


In [9]:
from huggingface_hub import HfApi
import os

repo_name = "suneeln-duke/test-onnx"

hf = HfApi()

hf.create_repo(repo_name, exist_ok=True, private=False)

model_path = "test_lr.onnx"

hf.upload_file(
    repo_id=repo_name, 
    path_or_fileobj=model_path, 
    token=hf_token,
    path_in_repo="test_lr.onnx",
    repo_type="model")

test_lr.onnx:   0%|          | 0.00/670 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/suneeln-duke/test-onnx/commit/e98db38f8a977b718e0d9c6ab4e7afebb3e7a317', commit_message='Upload test_lr.onnx with huggingface_hub', commit_description='', oid='e98db38f8a977b718e0d9c6ab4e7afebb3e7a317', pr_url=None, repo_url=RepoUrl('https://huggingface.co/suneeln-duke/test-onnx', endpoint='https://huggingface.co', repo_type='model', repo_id='suneeln-duke/test-onnx'), pr_revision=None, pr_num=None)

In [14]:
from huggingface_hub import hf_hub_download
import onnxruntime as ort
import numpy as np

# Step 1: Download the ONNX model from Hugging Face
model_path = hf_hub_download(repo_id="suneeln-duke/test-onnx", filename="test_lr.onnx")

# Step 2: Load the ONNX model
session = ort.InferenceSession(model_path)

# Step 3: Prepare input data (modify based on your model)
input_data = np.array([[5.1, 3.5, 1.4, 0.2]], dtype=np.float32)  # Example input

# Step 4: Get input and output names dynamically
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# Step 5: Run inference
predictions = session.run([output_name], {input_name: input_data})

# Step 6: Print predictions
print("Predictions:", predictions)


Predictions: [array([0], dtype=int64)]
